In [1]:
# Print your Emory ID here
print("Emory ID: 2593454")

Emory ID: 2593454


In [2]:
import pandas as pd
data=pd.read_csv("passwords.csv")


1. An online password attack is when someone tries to hack your account by simply trying a very large number of username/password combinations to access your account. For each `password` in our dataset, the `value` column shows the amount of time it is estimated to take an “online password attack” to hack your account. The column `time_unit` shows the units of that time value (e.g., hours, days, years, etc.). It would be much nicer if our values were of the same units so we can more easily compare the “online password guessing time” for each password. So your first task is to convert all of the values to units of hours (assume the conversion units provided below, e.g., 1 day is 24 hours, 1 week is 168 hours, etc).

In [5]:
units = {
    "hours": 1,
    "seconds": 1 / 3600,
    "minutes": 1 / 60,
    "days": 24,
    "weeks": 168,
    "months": 720,
    "years": 8760,
}

data['value_in_hours'] = data.apply(
    lambda row: row['value'] * units[row['time_unit']] if pd.notnull(row['value']) and row['time_unit'] in units else None, 
    axis=1
)

2. Are there any missing values in the dataset? If so, how many? After you have counted the, please drop them from the dataset.

In [11]:
missing = data.isnull().sum().sum()
print(missing)
data2= data.dropna()

70


3. Which password category has the highest average strength?


In [19]:
data3 = data2.groupby('category')['strength'].mean()
c = data3.idxmax()
display(c)

'nerdy-pop'

4. What is the rank of the password 'qwerty'?

In [25]:
data4=data2.query("password=='qwerty'")
display(data4['rank'])

4    5.0
Name: rank, dtype: float64

5. How many passwords have more than 8 characters?

In [27]:
data5 = data2.query("password.str.len() > 8")
count = len(data5)
display(count)

1

6. How many passwords do not contain a number?

In [29]:
data6 = data2.query("~password.str.contains('\d')")
count = len(data6)
display(count)

446

7. What is the average time in hours needed to crack these passwords that begin with `123`? How does this compare to the average of all passwords in the dataset?

In [32]:
data7 = data2[data2['password'].str.startswith('123')]
time_123 = data7['value_in_hours'].mean()
time_all = data2['value_in_hours'].mean()
print(f"avg time beginning with 123:{time_123}")
print(f"avg time of all passwords:{time_all}")

avg time beginning with 123:107.30332438271606
avg time of all passwords:13917.565247227776


8. What percentage of passwords are of the `simple-alphanumeric` category?

In [39]:
data8 = data2[data2['category'] == 'simple-alphanumeric'].shape[0]
total = data2.shape[0]
pct = (data8 / total) * 100
print(f"{pct}%")

12.2%


9. How many passwords have a strength rating below 5? What is their percentage?

In [43]:
data9=data2.query("strength<5")
pct=(data9.shape[0]/data2.shape[0])*100
print(f"{pct}%")

14.6%


10. What is the most common password in the dataset?

In [44]:
print(data2['password'].mode()[0])

1111


11. How many unique passwords are there in each category, and what is their average length?

In [45]:
data11 = data2.groupby('category').agg(
    unique_passwords=('password', 'nunique'),
    avg_length=('password', lambda x: x.str.len().mean())
)

print(data11)


                     unique_passwords  avg_length
category                                         
animal                             29    6.206897
cool-macho                         79    6.253165
fluffy                             44    5.795455
food                               11    6.090909
name                              183    6.218579
nerdy-pop                          30    6.633333
password-related                   15    6.333333
rebellious-rude                    11    6.363636
simple-alphanumeric                61    5.934426
sport                              37    6.513514


12. What is the median rank and strength for passwords of each length?

In [46]:
data2['p_len'] = data2['password'].str.len()

# Group by password length and calculate the median rank and strength
med = data2.groupby('p_len').agg(
    med_rank=('rank', 'median'),
    med_strength=('strength', 'median')
)

print(med)

       med_rank  med_strength
p_len                        
4         326.0           6.0
5         343.0           7.0
6         231.0           7.0
7         222.0           8.0
8         286.0           8.0
9          49.0           4.0


C:\Users\24981\AppData\Local\Temp\ipykernel_11896\3699921222.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['p_len'] = data2['password'].str.len()


13. Add a column `strength_label` with values `weak` if strength is below 5, else `strong`. Use a lambda function.

In [49]:
data2['strength_label'] = data2['strength'].apply(lambda x: 'weak' if x < 5 else 'strong')
display(data2[['strength','strength_label']] )

C:\Users\24981\AppData\Local\Temp\ipykernel_11896\280135091.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['strength_label'] = data2['strength'].apply(lambda x: 'weak' if x < 5 else 'strong')


,strength,strength_label
0,8.0,strong
1,4.0,weak
2,4.0,weak
3,4.0,weak
4,8.0,strong
...,...,...
495,6.0,strong
496,9.0,strong
497,7.0,strong
498,7.0,strong


14. Create a column `high_value` with a value of `True` if value is in the top 10% of values in the dataset.

In [52]:
bd = data2['value_in_hours'].quantile(0.90)
data2['high_value'] = data2['value_in_hours'] >= bd
print(bd)
display(data2[['value_in_hours','high_value']])

60531.6


C:\Users\24981\AppData\Local\Temp\ipykernel_11896\230963809.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data2['high_value'] = data2['value_in_hours'] >= bd


,value_in_hours,high_value
0,60531.600000,True
1,0.308667,False
2,30.960000,False
3,0.003086,False
4,89.280000,False
...,...,...
495,89.280000,False
496,60531.600000,True
497,2296.800000,False
498,89.280000,False


15. Join all passwords within each `category` into a single string.

In [54]:
data15 = data2.groupby('category')['password'].agg(lambda x: '_'.join(x))

display(data15)

category
animal                 dragon_monkey_bigdog_falcon_phoenix_tigers_chi...
cool-macho             mustang_shadow_master_harley_hunter_ranger_bus...
fluffy                 love_sunshine_silver_orange_ginger_summer_prin...
food                   pepper_cheese_coffee_banana_butter_muffin_hotd...
name                   michael_jennifer_jordan_superman_thomas_tigger...
nerdy-pop              starwars_computer_merlin_matrix_snoopy_boomer_...
password-related       password_letmein_test_pass_access_gateway_plea...
rebellious-rude        biteme_freedom_secret_whatever_badboy_sexsex_b...
simple-alphanumeric    123456_12345678_1234_qwerty_12345_696969_abc12...
sport                  baseball_football_soccer_hockey_dallas_yankees...
Name: password, dtype: object

**Bonus question 01**: What is the correlation between password length and strength? Create a scatter plot with regression line.

**Bonus question 02**: Find all passwords with palindromes.

In [55]:
data17=[]
for a in data2['password']:
    if a == a[::-1]:
        data17.append(a)
display(data17)


['111111',
 '666666',
 '1111',
 'aaaaaa',
 'xxxxxx',
 'hannah',
 '11111111',
 '7777777',
 '222222',
 'xxxxxxxx',
 '777777',
 '88888888',
 '2112',
 '7777',
 'zzzzzz',
 '555555',
 '2222',
 '4444',
 '11111',
 '123321',
 '999999',
 '333333',
 'xxxx',
 '888888']